# Playing around with the embeddings

In [48]:
import os
import umap
from umap.umap_ import UMAP
import torch
import pickle
import numpy as np
from tqdm import tqdm
import plotly.graph_objs as go
import plotly.io as pio
from datasets import load_dataset
from transformers import GPTJForCausalLM, AutoTokenizer
import pandas as pd


SUBSAMPLE_N = 10
SAVEDIR = './data'

In [14]:
def make_sphere_data(model_strings):
    # if os.path.exists('./data/figure_4_sphere_data.pkl'):
    #     print('Found cached sphere data')
    #     return

    generations_by_model_string = {}
    pre_umap = []
    for model_string in model_strings:
        save_string = model_string.replace('/', '_')

        with open('./data/' + save_string + '_fig4_embeddings.pkl', 'rb') as f:
            all_embs = pickle.load(f)
            for prompt in all_embs:
                for sequence in prompt:
                    sequence = np.squeeze(sequence)
                    sequence = np.stack(sequence).astype('float64')
                    sequence = sequence / np.linalg.norm(sequence, axis=1, keepdims=True)
                    sequence = np.cumsum(sequence, axis=0) / np.arange(1, sequence.shape[0]+1).reshape(-1, 1)
                    pre_umap.append(np.stack(sequence))
            generations_by_model_string[model_string] = all_embs

    pre_umap = np.concatenate(pre_umap, axis=0)
    # sphere_mapper = umap.UMAP(output_metric='haversine').fit(pre_umap)
    sphere_mapper = UMAP(output_metric='haversine').fit(pre_umap)
    x = np.sin(sphere_mapper.embedding_[:, 0]) * np.cos(sphere_mapper.embedding_[:, 1])
    y = np.sin(sphere_mapper.embedding_[:, 0]) * np.sin(sphere_mapper.embedding_[:, 1])
    z = np.cos(sphere_mapper.embedding_[:, 0])

    pointer = 0
    sphere_data_by_model_string = {model_string: [] for model_string in model_strings}
    for model_string in model_strings:
        for prompt in generations_by_model_string[model_string]:
            cur_prompt_continuations = []
            for sequence in prompt:
                cur_generation_points = []
                for _ in sequence:
                    cur_generation_points.append([x[pointer], y[pointer], z[pointer]])
                    pointer += 1
                cur_prompt_continuations.append(cur_generation_points)
            sphere_data_by_model_string[model_string].append(cur_prompt_continuations)

    return sphere_data_by_model_string
    # with open('./data/figure_4_sphere_data.pkl', 'wb') as f:
    #     pickle.dump(sphere_data_by_model_string, f)
    # with open('./data/figure_4_sphere_data2.pkl', 'wb') as f:
    #     pickle.dump(sphere_data_by_model_string, f)


def get_data(model_string):
    save_string = model_string.replace('/', '_')
    if os.path.exists('./data/' + save_string + '_fig4_embeddings.pkl'):
        print('found cached embeddings for model: ', save_string)
        with open('./data/' + save_string + '_fig4_embeddings.pkl', 'rb') as f:
            d = pickle.load(f)
            # return [e for e in d[:10]]
            return [e for e in d[:SUBSAMPLE_N]] # TODO: See if this was right change

    print('loading model')
    model = GPTJForCausalLM.from_pretrained(model_string,
                                            output_hidden_states=True,
                                            torch_dtype=torch.float16,
                                            low_cpu_mem_usage=True)
    model.cuda()
    print('loading tokenizer')
    tokenizer = AutoTokenizer.from_pretrained(model_string)
    print('loading dataset')
    ds = load_dataset('allenai/prosocial-dialog', split='test')
    print('formatting dataset')

    chosen_ds = [e['context'] for e in ds][:SUBSAMPLE_N]
    # chosen_ds = [e['context'] for e in ds]
    all_embeddings = []
    all_generations = []
    for e in tqdm(chosen_ds):
        input = tokenizer(e, return_tensors='pt', truncation=True, max_length=256)
        input = {k: v.cuda() for k, v in input.items()}


        n_new_tokens = 20
        cur_embeddings = []
        cur_generations = []
        for i in range(5):
            generated_tokens = model.generate(input['input_ids'],
                                              attention_mask=input['attention_mask'],
                                              max_length=input['input_ids'].shape[1] + n_new_tokens,
                                              min_new_tokens=10,
                                              num_beams=5,
                                              repetition_penalty=2.0,
                                              do_sample=True,
                                              num_return_sequences=1)


            generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
            cur_generations.append(generated_text)

            # Concatenate the new tokens to the original input_ids
            extended_input_ids = torch.cat((input['input_ids'], generated_tokens[:, -n_new_tokens:]), dim=1)
            extended_attention_mask = torch.cat(
                (input["attention_mask"], torch.ones_like(generated_tokens[:, -n_new_tokens:]).cuda()), dim=1)

            extended_input = {"input_ids": extended_input_ids, "attention_mask": extended_attention_mask}

            y = model(**extended_input)
            embedding = y.hidden_states[0].detach().cpu().numpy()[0]
            cur_embeddings.append(embedding)
            cur_generations.append(generated_text)

        cur_embeddings = np.stack(cur_embeddings)
        all_embeddings.append(cur_embeddings)
        all_generations.append(cur_generations)


    with open('./data/' + save_string + '_fig4_embeddings.pkl', 'wb') as f:
        pickle.dump(all_embeddings, f)
    with open('./data/' + save_string + '_fig4_generations.pkl', 'wb') as f:
        pickle.dump(all_generations, f)

    model.cpu()
    del model

# if __name__ == '__main__':
_ = get_data("EleutherAI/gpt-j-6B")
_ = get_data("reciprocate/ppo_hh_gpt-j")
sphere_data = make_sphere_data(["EleutherAI/gpt-j-6B", "reciprocate/ppo_hh_gpt-j"])

found cached embeddings for model:  EleutherAI_gpt-j-6B
found cached embeddings for model:  reciprocate_ppo_hh_gpt-j


/usr/local/lib/python3.10/dist-packages/umap/spectral.py:260: UserWarning: WARNING: spectral initialisation failed! The eigenvector solver
failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!
  warn(


In [38]:
print(len(sphere_data['EleutherAI/gpt-j-6B'][2][3]))

print(np.array(sphere_data['EleutherAI/gpt-j-6B']).shape)

# sphere_data['reciprocate/ppo_hh_gpt-j']

31
(10, 5)


/tmp/ipykernel_6438/3554192940.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(sphere_data['EleutherAI/gpt-j-6B']).shape)


In [52]:
model_strings = ["EleutherAI/gpt-j-6B", "reciprocate/ppo_hh_gpt-j"]
tokenizer1 = AutoTokenizer.from_pretrained(model_strings[0])
tokenizer2 = AutoTokenizer.from_pretrained(model_strings[1])

generations_by_model_string = {}
for model_string in model_strings:
    save_string = model_string.replace('/', '_')
    with open('./data/' + save_string + '_fig4_generations.pkl', 'rb') as f:
        generations_by_model_string[model_string] = pickle.load(f)

model_strings_df = []
labels_df = []
coordinates_df = []
prompt_or_continuation_df = []
index_number_df = []

for sample_idx in range(SUBSAMPLE_N):
    all_traces = []
    for i, coordinates1 in enumerate(sphere_data[model_strings[0]][sample_idx]):
        labels1 = tokenizer1.tokenize(generations_by_model_string[model_strings[0]][sample_idx][i])

        coordinates2 = sphere_data[model_strings[1]][sample_idx][i]
        labels2 = tokenizer2.tokenize(generations_by_model_string[model_strings[1]][sample_idx][i])

        k = 0
        for j, t in enumerate(labels1):
            if labels2[j] == t:
                k +=1
            else:
                break

        if not i:
            model_strings_df.append(model_strings[0])
            labels_df.append(labels1[:k])
            coordinates_df.append(coordinates1[:k])
            prompt_or_continuation_df.append("prompt")
            index_number_df.append(i)

            model_strings_df.append(model_strings[1])
            labels_df.append(labels2[:k])
            coordinates_df.append(coordinates2[:k])
            prompt_or_continuation_df.append("prompt")
            index_number_df.append(i)

        model_strings_df.append(model_strings[0])
        labels_df.append(labels1[k:])
        coordinates_df.append(coordinates1[k:])
        prompt_or_continuation_df.append("continuation")
        index_number_df.append(i)

        model_strings_df.append(model_strings[1])
        labels_df.append(labels2[k:])
        coordinates_df.append(coordinates2[k:])
        prompt_or_continuation_df.append("continuation")
        index_number_df.append(i)

df = pd.DataFrame(data={'model_strings': model_strings_df, 'labels': labels_df, 'coordinates': coordinates_df, 'prompt_or_continuation': prompt_or_continuation_df, 'index_number': index_number_df})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [53]:
df.to_csv("tmp.csv")